# Trail Study

This notebook studies the implementation of the Trail mark. The idea of a trail is to construct a line with a varying width.

In [1]:
using Pkg
Pkg.activate("../.")
using TestEnv
TestEnv.activate()
using Vizagrams
using Transducers
using LinearAlgebra

  Activating project at `~/Documents/GitHub/Vizagrams.jl`
Precompiling Vizagrams
  ✓ Vizagrams
  1 dependency successfully precompiled in 3 seconds. 154 already precompiled.


Consider a collection of points `pts` and weights which we will represent as a radius of a circle.

In [2]:
pts= [[0,0],[1.7,1],[2,0],[2,1],[3,1],[3,0]]
ws = 1/10*[1,0.5,0.4,0.9,0.5,0.5]

d = mapreduce(x->S(:opacity=>0.2)Circle(c=x[1],r=x[2]),+,zip(pts,ws)) + Line(pts)
draw(d)

<svg height="300" viewBox="-0.11983333333333333 -1.1098333333333332 3.1896666666666667 1.2296666666666667" width="794.1176470588235" preserveAspectRatio="xMidYMid meet" xmlns="http://www.w3.org/2000/svg" version="1.1"><circle cy="0.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.1" transform="scale(1,-1)" cx="0.0" /><circle cy="1.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.050000000000000044" transform="scale(1,-1)" cx="1.7" /><circle cy="0.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.040000000000000036" transform="scale(1,-1)" cx="2.0" /><circle cy="1.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.08999999999999997" transform="scale(1,-1)" cx="2.0" /><circle cy="1.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.04999999999999982" transform="scale(1,-1)" cx="3.0" /><circle cy="0.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.04999999999999982" transform="scale(1,-1)" cx="3.0" /><polyline points="0.0 0.0 1.7 1.0 2.0 0.0 2.0 1.0 3.0 1.0 3.0 0.0 " style="vector-effect: non-scaling-stroke; stroke: black; fill: none; " transform="scale(1,-1)" /></svg>

How do we now draw a trail? We start by computing the angle for each segment, and then the points perpendicular to the lines that touch each circle.

In [3]:
angles = collect(
    Map(v -> atan(v[2], v[1]))(Map(x -> x[2] - x[1])(Partition(2; step=1)(pts)))
);

In [4]:
function compute_perpendicular_points(pts,ang=π/2)
    perps = []
    for i in 1:length(pts)
        if i == 1
            anglein = angles[i] + ang
            vin = pts[i] + R(anglein)([ws[i], 0])
            push!(perps, vin)
            continue
        elseif i == length(pts)
            anglein = angles[i-1] + ang
            vin = pts[i] + R(anglein)([ws[i], 0])
            push!(perps, vin)
            continue
        end
        anglein = angles[i-1] + ang
        angleout = angles[i] + ang
        vin = pts[i] + R(anglein)([ws[i], 0])
        vout = pts[i] + R(angleout)([ws[i], 0])
        push!(perps, vin,vout)
    end
    return perps
end

pts_pos = compute_perpendicular_points(pts)
pts_neg = compute_perpendicular_points(pts,-π/2)
points = mapreduce(x->S(:fill=>:green)Circle(c=x,r=0.01),+,pts_pos) + mapreduce(x->S(:fill=>:red)Circle(c=x,r=0.01),+,pts_neg)

dpts = d + points
draw(dpts)

<svg height="300" viewBox="-0.12000000000000001 -1.12 3.1999999999999997 1.2400000000000002" width="789.9999999999999" preserveAspectRatio="xMidYMid meet" xmlns="http://www.w3.org/2000/svg" version="1.1"><circle cy="0.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.1" transform="scale(1,-1)" cx="0.0" /><circle cy="1.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.050000000000000044" transform="scale(1,-1)" cx="1.7" /><circle cy="0.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.040000000000000036" transform="scale(1,-1)" cx="2.0" /><circle cy="1.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.08999999999999997" transform="scale(1,-1)" cx="2.0" /><circle cy="1.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.04999999999999982" transform="scale(1,-1)" cx="3.0" /><circle cy="0.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.04999999999999982" transform="scale(1,-1)" cx="3.0" /><polyline points="0.0 0.0 1.7 1.0 2.0 0.0 2.0 1.0 3.0 1.0 3.0 0.0 " style="vector-effect: non-scaling-stroke; stroke: black; fill: none; " transform="scale(1,-1)" /><circle cy="0.08619342151577696" style="vector-effect: non-scaling-stroke; fill: green; " r="0.010000000000000002" transform="scale(1,-1)" cx="-0.05070201265633939" /><circle cy="1.0430967107578886" style="vector-effect: non-scaling-stroke; fill: green; " r="0.010000000000000009" transform="scale(1,-1)" cx="1.6746489936718303" /><circle cy="1.0143673942783173" style="vector-effect: non-scaling-stroke; fill: green; " r="0.010000000000000009" transform="scale(1,-1)" cx="1.7478913142610575" /><circle cy="0.011493915422653824" style="vector-effect: non-scaling-stroke; fill: green; " r="0.009999999999999787" transform="scale(1,-1)" cx="2.038313051408846" /><circle cy="4.8985871965894135e-18" style="vector-effect: non-scaling-stroke; fill: green; " r="0.010000000000000009" transform="scale(1,-1)" cx="1.96" /><circle cy="1.0" style="vector-effect: non-scaling-stroke; fill: green; " r="0.010000000000000009" transform="scale(1,-1)" cx="1.91" /><circle cy="1.09" style="vector-effect: non-scaling-stroke; fill: green; " r="0.009999999999999898" transform="scale(1,-1)" cx="2.0" /><circle cy="1.05" style="vector-effect: non-scaling-stroke; fill: green; " r="0.009999999999999787" transform="scale(1,-1)" cx="3.0" /><circle cy="1.0" style="vector-effect: non-scaling-stroke; fill: green; " r="0.009999999999999787" transform="scale(1,-1)" cx="3.05" /><circle cy="0.0" style="vector-effect: non-scaling-stroke; fill: green; " r="0.009999999999999787" transform="scale(1,-1)" cx="3.05" /><circle cy="-0.08619342151577696" style="vector-effect: non-scaling-stroke; fill: red; " r="0.010000000000000002" transform="scale(1,-1)" cx="0.050702012656339404" /><circle cy="0.9569032892421115" style="vector-effect: non-scaling-stroke; fill: red; " r="0.010000000000000009" transform="scale(1,-1)" cx="1.7253510063281696" /><circle cy="0.9856326057216828" style="vector-effect: non-scaling-stroke; fill: red; " r="0.010000000000000009" transform="scale(1,-1)" cx="1.6521086857389424" /><circle cy="-0.011493915422653829" style="vector-effect: non-scaling-stroke; fill: red; " r="0.010000000000000009" transform="scale(1,-1)" cx="1.961686948591154" /><circle cy="0.0" style="vector-effect: non-scaling-stroke; fill: red; " r="0.009999999999999787" transform="scale(1,-1)" cx="2.04" /><circle cy="1.0" style="vector-effect: non-scaling-stroke; fill: red; " r="0.009999999999999787" transform="scale(1,-1)" cx="2.09" /><circle cy="0.91" style="vector-effect: non-scaling-stroke; fill: red; " r="0.009999999999999898" transform="scale(1,-1)" cx="2.0" /><circle cy="0.95" style="vector-effect: non-scaling-stroke; fill: red; " r="0.009999999999999787" transform="scale(1,-1)" cx="3.0" /><circle cy="1.0" style="vector-effect: non-scaling-stroke; fill: red; " r="0.009999999999999787" transform="scale(1,-1)" cx="2.95" /><circle cy="-6.123233995736766e-1

We have the main points to construct our trail. Let us first draw a "sketch" by drawing the possible line segments.

In [5]:
lsp =S(:stroke=>:green)reduce(+,pts_pos |> Partition(2,step=2) |> Map(x->Line(x)) |> collect)
lsn =S(:stroke=>:red)reduce(+,pts_neg |> Partition(2,step=2) |> Map(x->Line(x)) |> collect)

draw(dpts +lsn + lsp)

<svg height="300" viewBox="-0.12000000000000001 -1.12 3.1999999999999997 1.2400000000000002" width="789.9999999999999" preserveAspectRatio="xMidYMid meet" xmlns="http://www.w3.org/2000/svg" version="1.1"><circle cy="0.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.1" transform="scale(1,-1)" cx="0.0" /><circle cy="1.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.050000000000000044" transform="scale(1,-1)" cx="1.7" /><circle cy="0.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.040000000000000036" transform="scale(1,-1)" cx="2.0" /><circle cy="1.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.08999999999999997" transform="scale(1,-1)" cx="2.0" /><circle cy="1.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.04999999999999982" transform="scale(1,-1)" cx="3.0" /><circle cy="0.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.04999999999999982" transform="scale(1,-1)" cx="3.0" /><polyline points="0.0 0.0 1.7 1.0 2.0 0.0 2.0 1.0 3.0 1.0 3.0 0.0 " style="vector-effect: non-scaling-stroke; stroke: black; fill: none; " transform="scale(1,-1)" /><circle cy="0.08619342151577696" style="vector-effect: non-scaling-stroke; fill: green; " r="0.010000000000000002" transform="scale(1,-1)" cx="-0.05070201265633939" /><circle cy="1.0430967107578886" style="vector-effect: non-scaling-stroke; fill: green; " r="0.010000000000000009" transform="scale(1,-1)" cx="1.6746489936718303" /><circle cy="1.0143673942783173" style="vector-effect: non-scaling-stroke; fill: green; " r="0.010000000000000009" transform="scale(1,-1)" cx="1.7478913142610575" /><circle cy="0.011493915422653824" style="vector-effect: non-scaling-stroke; fill: green; " r="0.009999999999999787" transform="scale(1,-1)" cx="2.038313051408846" /><circle cy="4.8985871965894135e-18" style="vector-effect: non-scaling-stroke; fill: green; " r="0.010000000000000009" transform="scale(1,-1)" cx="1.96" /><circle cy="1.0" style="vector-effect: non-scaling-stroke; fill: green; " r="0.010000000000000009" transform="scale(1,-1)" cx="1.91" /><circle cy="1.09" style="vector-effect: non-scaling-stroke; fill: green; " r="0.009999999999999898" transform="scale(1,-1)" cx="2.0" /><circle cy="1.05" style="vector-effect: non-scaling-stroke; fill: green; " r="0.009999999999999787" transform="scale(1,-1)" cx="3.0" /><circle cy="1.0" style="vector-effect: non-scaling-stroke; fill: green; " r="0.009999999999999787" transform="scale(1,-1)" cx="3.05" /><circle cy="0.0" style="vector-effect: non-scaling-stroke; fill: green; " r="0.009999999999999787" transform="scale(1,-1)" cx="3.05" /><circle cy="-0.08619342151577696" style="vector-effect: non-scaling-stroke; fill: red; " r="0.010000000000000002" transform="scale(1,-1)" cx="0.050702012656339404" /><circle cy="0.9569032892421115" style="vector-effect: non-scaling-stroke; fill: red; " r="0.010000000000000009" transform="scale(1,-1)" cx="1.7253510063281696" /><circle cy="0.9856326057216828" style="vector-effect: non-scaling-stroke; fill: red; " r="0.010000000000000009" transform="scale(1,-1)" cx="1.6521086857389424" /><circle cy="-0.011493915422653829" style="vector-effect: non-scaling-stroke; fill: red; " r="0.010000000000000009" transform="scale(1,-1)" cx="1.961686948591154" /><circle cy="0.0" style="vector-effect: non-scaling-stroke; fill: red; " r="0.009999999999999787" transform="scale(1,-1)" cx="2.04" /><circle cy="1.0" style="vector-effect: non-scaling-stroke; fill: red; " r="0.009999999999999787" transform="scale(1,-1)" cx="2.09" /><circle cy="0.91" style="vector-effect: non-scaling-stroke; fill: red; " r="0.009999999999999898" transform="scale(1,-1)" cx="2.0" /><circle cy="0.95" style="vector-effect: non-scaling-stroke; fill: red; " r="0.009999999999999787" transform="scale(1,-1)" cx="3.0" /><circle cy="1.0" style="vector-effect: non-scaling-stroke; fill: red; " r="0.009999999999999787" transform="scale(1,-1)" cx="2.95" /><circle cy="-6.123233995736766e-1

We see that some segments cross each other, while others do not touch. Consider, for example, the second circle in the drawing (from left to right).
In this circle, the green lines do not match, while the red ones cross. We want extend the green lines until they reach each other, and we want to
cut the red lines so they stop as soon as they touch.

In [6]:
"""
compute_point_line_extension(r::Real, a::Line, b::Line)

Helper function. It takes two lines and the radius, and extend them
by 5 times the radius. The first line is extended by the final point,
while the second is extended by the first. If the two lines intersect after the extension,
it then returns the intersection point. Otherwise, it returns the two
non-extended points.
"""
function compute_point_line_extension(r::Real, a::Line, b::Line)
    ae = a.pts[2]+5r*normalize(a.pts[2] - a.pts[1])
    ae = Line([a.pts[2],ae])
    
    be = b.pts[1]-5r*normalize(b.pts[2] - b.pts[1])
    
    be = Line([b.pts[1],be])
    flag, ipt = intersects(ae,be)
    if flag
        return [ipt]
    end
    return [a.pts[2], b.pts[1]]
end


"""
compute_extension_or_cutoff(r::Real,a::Line,b::Line,pt)

Given two lines and the radius, it computes the intersection point, which
can be achieved by cutting off the lines, or by extending them and finding the intersection.
"""
function compute_extension_or_cutoff(r::Real,a::Line,b::Line,pt)
    flag, ipt = intersects(a,b)
    if flag
        return [ipt]
    end
    return compute_point_line_extension(r,a,b)
end

compute_extension_or_cutoff

In [35]:
ls_pos = pts_pos |> Partition(4,step=2) |> Map(x->[Line([x[1],x[2]]),Line([x[3],x[4]])]) |> collect
ls_neg = pts_neg |> Partition(4,step=2) |> Map(x->[Line([x[1],x[2]]),Line([x[3],x[4]])]) |> collect
new_pts_neg=[]
new_pts_pos=[]
for i in 1:length(ls_neg)
    npt = compute_extension_or_cutoff(ws[i+1],ls_neg[i]...,pts[i+1])
    push!(new_pts_neg,npt...)
    
    ppt = compute_extension_or_cutoff(ws[i+1],ls_pos[i]...,pts[i+1])
    push!(new_pts_pos,ppt...)
end

new_pts_neg = [pts_neg[begin],new_pts_neg...,pts_neg[end]]
new_pts_pos = [pts_pos[begin],new_pts_pos...,pts_pos[end]]

ncs = mapreduce(x->S(:fill=>:red)Circle(c=x,r=0.01),+,new_pts_neg)
pcs = mapreduce(x->S(:fill=>:green)Circle(c=x,r=0.01),+,new_pts_pos)
idraw(d + ncs+pcs + S(:stroke=>:green)Line(new_pts_pos)+ S(:stroke=>:red)Line(new_pts_neg))

HTML{String}("    <svg height=\"300\" viewBox=\"-0.12006312625250502 -1.1238507014028059 3.20012625250501 1.243913827655311\" id=\"337e09c0-39ff-11ef-0a54-b57e2131899e\" width=\"787.5143584877388\" preserveAspectRatio=\"xMidYMid meet\" xmlns=\"http://www.w3.org/2000/svg\" version=\"1.1\"><circle cy=\"0.0\" style=\"vector-effect: non-scaling-stroke; opacity: 0.2; \" r=\"0.1\" transform=\"scale(1,-1)\" cx=\"0.0\" /><circle cy=\"1.0\" style=\"vector-effect: non-scaling-stroke; opacity: 0.2; \" r=\"0.050000000000000044\" transform=\"scale(1,-1)\" cx=\"1.7\" /><circle cy=\"0.0\" style=\"vector-effect: non-scaling-stroke; opacity: 0.2; \" r=\"0.040000000000000036\" transform=\"scale(1,-1)\" cx=\"2.0\" /><circle cy=\"1.0\" style=\"vector-effect: non-scaling-stroke; opacity: 0.2; \" r=\"0.08999999999999997\" transform=\"scale(1,-1)\" cx=\"2.0\" /><circle cy=\"1.0\" style=\"vector-effect: non-scaling-stroke; opacity: 0.2; \" r=\"0.04999999999999982\" transform=\"scale(1,-1)\" cx=\"3.0\" /><circle cy=\"0.0\" style=\"vector-effect: non-scaling-stroke; opacity: 0.2; \" r=\"0.04999999999999982\" transform=\"scale(1,-1)\" cx=\"3.0\" /><polyline points=\"0.0 0.0 1.7 1.0 2.0 0.0 2.0 1.0 3.0 1.0 3.0 0.0 \" style=\"vector-effect: non-scaling-stroke; stroke: black; fill: none; \" transform=\"scale(1,-1)\" /><circle cy=\"-0.08619342151577696\" style=\"vector-effect: non-scaling-stroke; fill: red; \" r=\"0.010000000000000002\" transform=\"scale(1,-1)\" cx=\"0.050702012656339404\" /><circle cy=\"0.923330695741706\" style=\"vector-effect: non-scaling-stroke; fill: red; \" r=\"0.010000000000000009\" transform=\"scale(1,-1)\" cx=\"1.67145158421269\" /><circle cy=\"-0.011493915422653829\" style=\"vector-effect: non-scaling-stroke; fill: red; \" r=\"0.010000000000000009\" transform=\"scale(1,-1)\" cx=\"1.961686948591154\" /><circle cy=\"0.0\" style=\"vector-effect: non-scaling-stroke; fill: red; \" r=\"0.009999999999999787\" transform=\"scale(1,-1)\" cx=\"2.04\" /><circle cy=\"0.9134268537074149\" style=\"vector-effect: non-scaling-stroke; fill: red; \" r=\"0.009999999999999787\" transform=\"scale(1,-1)\" cx=\"2.085671342685371\" /><circle cy=\"0.948\" style=\"vector-effect: non-scaling-stroke; fill: red; \" r=\"0.009999999999999787\" transform=\"scale(1,-1)\" cx=\"2.95\" /><circle cy=\"-6.123233995736766e-18\" style=\"vector-effect: non-scaling-stroke; fill: red; \" r=\"0.009999999999999787\" transform=\"scale(1,-1)\" cx=\"2.95\" /><circle cy=\"0.08619342151577696\" style=\"vector-effect: non-scaling-stroke; fill: green; \" r=\"0.010000000000000002\" transform=\"scale(1,-1)\" cx=\"-0.05070201265633939\" /><circle cy=\"1.07412087618904\" style=\"vector-effect: non-scaling-stroke; fill: green; \" r=\"0.010000000000000009\" transform=\"scale(1,-1)\" cx=\"1.7305873268857581\" /><circle cy=\"0.34075588964612247\" style=\"vector-effect: non-scaling-stroke; fill: green; \" r=\"0.010000000000000009\" transform=\"scale(1,-1)\" cx=\"1.9429622055176938\" /><circle cy=\"1.0937875751503008\" style=\"vector-effect: non-scaling-stroke; fill: green; \" r=\"0.010000000000000009\" transform=\"scale(1,-1)\" cx=\"1.9053106212424848\" /><circle cy=\"1.048\" style=\"vector-effect: non-scaling-stroke; fill: green; \" r=\"0.009999999999999787\" transform=\"scale(1,-1)\" cx=\"3.05\" /><circle cy=\"0.0\" style=\"vector-effect: non-scaling-stroke; fill: green; \" r=\"0.009999999999999787\" transform=\"scale(1,-1)\" cx=\"3.05\" /><polyline points=\"-0.05070201265633939 0.08619342151577696 1.7305873268857581 1.07412087618904 1.9429622055176938 0.34075588964612247 1.9053106212424848 1.0937875751503008 3.05 1.048 3.05 0.0 \" style=\"vector-effect: non-scaling-stroke; stroke: green; fill: none; \" transform=\"scale(1,-1)\" /><polyline points=\"0.050702012656339404 -0.08619342151577696 1.67145158421269 0.923330695741706 1.961686948591154 -0.011493915422653829 2.04 0.0 2.085671342685371 0.9134268537074149 2.95 0.948 2.95 -6.123233995736766e-18 \" style=\"vector-effect: non-scaling-stro

At last, we use these points to draw our trail. Let us compare with the version implemented in Vizagrams.

In [33]:
pts= [[0,0],[1.7,1],[2,0],[2,1],[3,1],[3,0]]
ws = 1/10*[1,0.5,0.4,0.9,0.5,0.5]
draw(S(:opacity=>0.1)Trail(pts,ws) + d)

<svg height="300" viewBox="-0.11989645958583836 -1.1136840347361392 3.1897929191716767 1.2335804943219775" width="791.5981198589892" preserveAspectRatio="xMidYMid meet" xmlns="http://www.w3.org/2000/svg" version="1.1"><polygon points="-0.05070201265633939 0.08619342151577696 1.7305873268857581 1.07412087618904 1.9429622055176938 0.34075588964612247 1.9053106212424848 1.0937875751503008 3.05 1.048 3.05 0.0 2.95 -6.123233995736766e-18 2.95 0.948 2.085671342685371 0.9134268537074149 2.04 0.0 1.961686948591154 -0.011493915422653829 1.67145158421269 0.923330695741706 0.050702012656339404 -0.08619342151577696 " style="vector-effect: non-scaling-stroke; opacity: 0.1; " transform="scale(1,-1)" /><circle cy="0.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.1" transform="scale(1,-1)" cx="0.0" /><circle cy="1.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.050000000000000044" transform="scale(1,-1)" cx="1.7" /><circle cy="0.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.040000000000000036" transform="scale(1,-1)" cx="2.0" /><circle cy="1.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.08999999999999997" transform="scale(1,-1)" cx="2.0" /><circle cy="1.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.04999999999999982" transform="scale(1,-1)" cx="3.0" /><circle cy="0.0" style="vector-effect: non-scaling-stroke; opacity: 0.2; " r="0.04999999999999982" transform="scale(1,-1)" cx="3.0" /><polyline points="0.0 0.0 1.7 1.0 2.0 0.0 2.0 1.0 3.0 1.0 3.0 0.0 " style="vector-effect: non-scaling-stroke; stroke: black; fill: none; " transform="scale(1,-1)" /></svg>

Also, note how this matches the line when we use a fixed width. 

In [46]:
pts =[[0,0],[1,3.8],[2,0],[3,0],[4,2]]
cs = mapreduce(x->S(:opacity=>0.2)Circle(c=x,r=0.1) + Circle(c=x,r=0.01),+,pts)
draw(S(:fillOpacity=>1.)Trail(pts=pts, ws=0.1))

<svg height="300" viewBox="-0.1682564146277275 -4.2644866149892335 4.3292480946291825 4.4360355758906955" width="292.5374540792716" preserveAspectRatio="xMidYMid meet" xmlns="http://www.w3.org/2000/svg" version="1.1"><polygon points="-0.09670745372626464 0.025449329927964382 1.0 4.19293765408777 2.077088856338887 0.10000000000000009 2.9381966011250107 0.1 3.9105572809000084 2.044721359549996 4.089442719099992 1.9552786404500042 3.0618033988749893 -0.1 1.9229111436611133 -0.1 1.0 3.40706234591223 0.09670745372626466 -0.025449329927964365 " style="vector-effect: non-scaling-stroke; fill-opacity: 1.0; " transform="scale(1,-1)" /></svg>